# Setup

## Imports & Settings

In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import itertools
import functools
import scipy
import scanpy as sc
import seaborn as sb
import pandas as pd
import numpy as np
import corescpy as cr

# Computing Resources
gpu = False
sc.settings.n_jobs = 8
# sc.settings.max_memory = 150

# Display
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(20, 20))

# Panel & Column Names (from Metadata & To Be Created)
panel = "TUQ97N"
# panel = "XR4UZH"
sub_dir_parent = None
# sub_dirs = None  # don't run analyses on objects in sub-directories
sub_dir_parent = "objects_cropped"
sub_dirs = ["mucosa", "submucosa", "myenteric_plexus",
            "smc_circular", "smc_longitudinal", "serosa"]
suffixes = [f"_{x}" for x in sub_dirs] if sub_dirs else None
capitalize_sample = True if panel == "TUQ97N" else False
# suffix = "_new"  # suffix for object h5ad file (to avoid overwrite)

# Samples/Runs
run = None  # just look for samples in all Xenium runs for the panel
# run = "CHO-001"  # run all from this run; so don't have to specify samples
samples = "all"  # use samples = "all" with run = something for all from run
# or run = None for all available samples
# samples = ["50006B"]
genes = {
    "CD Risk": ["LACC1", "LRRK2", "PTGER4"],
    "SnC Marker": ["CDKN2A", "CDKN1A", "TP53", "PLAUR"],
    "Apoptosis-Resistance": ["BCL2"],
    "SASP": ["IL4", "IL13", "IL1A", "CXCL8", "CCL2", "CEBPB", "NFKB1",
             "TGFB1", "IGFBP7"],
    "SASP-IL6": ["OSM", "IL6", "IL6ST"],
    "Fibrosis": ["SUCNR1", "CXCR4", "IL33", "IL23A",
                 "IL1B", "IL12", "IL17", "IL36", "TL1A", "IL11", "IFNG",
                 "IL22", "IL10", "IL21", "IL34", "CCL11"],
    "ER Stress/UPR": ["ATF4"],
    "Autophagy": ["MTOR", "ATG3", "ATG5", "ATG7", "ATG12", "ATG16L1", "SQSTM1",
                  "BECN1", "IRGM1", "ATG8", "ATG6"],
    "DNA Repair": ["ERCC1", "ERCC4"],
    "Healing": ["LCN2", "MMP9", "GREM1", "PDGFRA"],
    "Mixed": ["ICAM1"]
}   # genes of interest: can just specify list of genes
genes_dict_colors = [
    "#FF0000", "#0000FF", "#000000", "#FFFF00", "#D2B48C",
    "#FFC0CB", "#A52A2A", "#800080", "#008000", "#808080"
] if isinstance(genes, dict) else None  # colors for each "key" of genes dict

# Main Directories
usr_write_rel_path = f"{os.getlogin()}/data/shared-xenium-library"
d_hpc = "/mnt/cho_lab" if os.path.exists(
    "/mnt/cho_lab") else "/sc/arion/projects/untreatedIBD"  # HPC path
d_nfs = os.path.join(d_hpc, "bbdata2") if os.path.exists(os.path.join(
    d_hpc, "bbdata2")) else os.path.join(
        d_hpc, "chobiolab-core/shared-xenium-library")
d_usr = os.path.join(d_hpc, "disk2", usr_write_rel_path) if os.path.exists(
    os.path.join(d_hpc, "disk2")) else os.path.join(d_hpc, usr_write_rel_path)
d_obj = d_usr  # CHANGE IF WRITING & PULLING OBJECTS FROM DIFFERENT PATHS
d_img = os.path.join(d_hpc, f"cache/tissue-registry/xenium/{panel}") if (
    "arion" in d_hpc) else os.path.join(d_hpc, f"bbdata1/xenium/{panel}")

# Construct Directories (Less Likely to Need Changes)
# Mirror my file/directory tree in the `d_usr` directory
obj_ext = ".h5ad"  # extension for processed objects
out_subdir_markers = "find_markers"  # sub-directory under out_dir for markers
out_obj = os.path.join(d_obj, f"outputs/{panel}/nebraska")
out_dir = os.path.join(d_usr, f"outputs/{panel}/nebraska")  # to save objects
out_plot = None if out_dir is None else os.path.join(
    out_dir, "plots")  # plot output directory
file_mdf = os.path.join(d_usr, f"samples_{panel}.csv")  # metadata file path
file_a = None  # don't map manual annotations
file_a = os.path.join(out_dir, "annotation_dictionaries/annotations_all.xlsx")
print(f"\n\n\n{'=' * 80}\nDirectories\n{'=' * 80}\n\nHPC Entry Point (Cho): "
      f"{d_hpc}\nData: {d_nfs}\nMetadata: {file_mdf}\nImages: {d_img}\n"
      f"Object/Outputs:\n\t{out_dir} (objects)\n\t{out_plot} (plots)\n\t"
      f"{os.path.join(out_dir, out_subdir_markers)} (markers)\n"
      f"Annotations: {file_a}\n\n\n")

# Leiden Column & Manual Annotation Mapping Options
col_leiden = f"leiden_res1pt5_dist0_npc30"
col_assignment = "bucket"  # column in annotation file whose labels to use
col_cluster = col_leiden if file_a is None else col_assignment
# col_assignment = "annotation"  # more specific cell types
suffix_ct = f"_{col_leiden}" + str(f"_{col_cluster}" if (
    col_cluster != col_leiden) else "")  # new out file suffix ~ cell type

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.




Directories

HPC Entry Point (Cho): /mnt/cho_lab
Data: /mnt/cho_lab/bbdata2
Metadata: /mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/samples_TUQ97N.csv
Images: /mnt/cho_lab/bbdata1/xenium/TUQ97N
Object/Outputs:
	/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska (objects)
	/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/plots (plots)
	/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/find_markers (markers)
Annotations: /mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/annotation_dictionaries/annotations_all.xlsx





## Setup

Get constants (e.g., column names in metadata), read metadata, create dictionary of clustering parameters (so can iterate across different clustering specifications to make multiple versions, e.g., at multiple resolutions) using `res_list`, `min_dist_list`, and `n_comps_list`, make any output directories (e.g., for processed objects, plots, find markers results, Xenium Explorer cluster files) if any don't exist yet, load data into objects, etc.

In [2]:
# Get/Set Constants
constants_dict = cr.get_panel_constants(panel_id=panel)
col_sample_id_o, col_sample_id, col_condition, col_inflamed, col_subject = [
    constants_dict[x] if x in constants_dict else None for x in [
        "col_sample_id_o", "col_sample_id", "col_condition",
        "col_inflamed", "col_subject"]]
col_stricture, key_stricture, col_f, col_tangram, col_segment, col_object = [
    constants_dict[x] if (x in constants_dict) else None for x in [
        "col_stricture", "key_stricture", "col_data_dir",
        "col_tangram", "col_segment", "col_object"]]

# Read Metadata
metadata = cr.pp.get_metadata_cho(
    d_nfs, file_mdf, panel_id=panel, samples=samples, run=run,
    capitalize_sample=capitalize_sample)  # get metadata
print("\n\n", metadata[list(set([
    col_sample_id_o, col_subject, col_condition, col_inflamed, col_stricture,
    col_segment]).intersection(metadata))])

# Annotation File
f_ann = pd.read_excel(file_a, index_col=[0, 1]).dropna(how="all").dropna(
    how="all", axis=1)
f_ann = f_ann.reset_index().astype({f_ann.index.names[1]: "int"}).astype({
    f_ann.index.names[1]: "string"}).set_index(f_ann.index.names)

# Final Setup
if out_plot is not None:
    os.makedirs(out_plot, exist_ok=True)  # ensure plot save path exists
kws_init = dict(col_sample_id=col_sample_id, col_subject=col_subject,
                col_cell_type=col_cluster)



                    sample_id   Condition stricture inflammation  subject_id
Sample                                                                     
Uninflamed-50403A2   50403A2  uninflamed        no   uninflamed       50403
Uninflamed-50403A1   50403A1  uninflamed        no   uninflamed       50403
Uninflamed-50336C     50336C  uninflamed        no   uninflamed       50336
Inflamed-50336B       50336B    inflamed        no     inflamed       50336
Stricture-50336A      50336A   stricture       yes     inflamed       50336
Stricture-50403C2    50403C2   stricture       yes     inflamed       50403
Stricture-50403C1    50403C1   stricture       yes     inflamed       50403
Inflamed-50403B       50403B    inflamed        no     inflamed       50403
Stricture-50217C      50217C   stricture       yes     inflamed       50217
Uninflamed-50217B     50217B  uninflamed        no   uninflamed       50217
Inflamed-50217A       50217A    inflamed        no     inflamed       50217
Stricture

# Analyze

The first clustering version (first specified in `res_list`) is the cell type column used by default in downstream analyses (because it was specified in `kws_init["col_cell_type"]` when creating the object and thus is stored in `self._columns["col_cell_type"]`). Specify `col_cell_type` as an argument in the following functions to use a different column.

## Main Objects

In [3]:
# Set Up Plot Dictionaries
fig_central, fig_neigh, fig_cooccur, fig_svg = {}, {}, {}, {}
fig_gex = {"spatial": {}, "heat": {}, "dot": {}}

# Iterate Samples
for x in metadata.index.values:
    print(f"\n\n{'=' * 80}\n{x}\n{'=' * 80}\n\n")

    # Load Data
    out = os.path.join(out_obj, x + suffix + obj_ext)  # object output path
    if not os.path.exists(out):
        print(f"Skipping {x}: {out} doesn't exist!")
        continue

    self = cr.Spatial(metadata.loc[x][col_f], library_id=x, **kws_init)
    self.update_from_h5ad(out)  # update with prior preprocessing
    for j in metadata.dropna(how="all", axis=1):  # add metadata to .obs
        self.rna.obs.loc[:, j] = str(metadata.loc[x][j])
    self.rna.obs.loc[:, col_object] = out  # path for processed object

    # Map Manual Annotations
    annot = f_ann.loc[f"{self._library_id}___{col_leiden}_dictionary.xlsx"]
    _ = self.annotate_clusters(
        annot[col_assignment], col_cell_type=col_leiden,
        col_annotation=col_cluster, copy=False)

    # Gene Expression Plots (Only GEX Data)
    goi = list(set(pd.unique(functools.reduce(lambda u, v: u + v, [
        genes[g] for g in genes])) if isinstance(genes, dict) else genes
                   ).intersection(self.rna.var_names))  # gene list
    fig_gex["spatial"][self._library_id] = self.plot_spatial(
        color=goi + [col_cluster], title=f"{self._library_id}{suffix}")
    fig_gex["dot"][self._library_id] = cr.pl.plot_dot(
        self.rna, "Annotation", genes, title=f"{self._library_id}{suffix}",
        genes_dict_colors=genes_dict_colors, vmin=0, vmax=10,
        percent="right", center=None)
    fig_gex["heat"][self._library_id] = cr.pl.plot_matrix(
        self.rna, "Annotation", genes, title=f"{self._library_id}{suffix}",
        genes_dict_colors=genes_dict_colors, vmin=0, vmax=10,
        percent="right", center=None)

    # Spatial Analyses
    _, fig_central[self._library_id] = self.calculate_centrality(
        figsize=(20, 10))
    _, fig_neigh[self._library_id] = self.calculate_neighborhood(
        figsize=(60, 30))
    _, fig_cooccur[self._library_id] = self.find_cooccurrence(
        figsize=(60, 20), kws_plot=dict(wspace=3))
    _, fig_svg[self._library_id] = self.find_svgs(
        genes=goi, method="moran", n_perms=10, kws_plot=dict(
            legend_fontsize="large"), figsize=(15, 15))

    # Write Plots
    if out_plot is not None:
        for v, u in zip(["central", "neighbor", "cooccur", "svg"], [
                fig_central, fig_neigh, fig_cooccur, fig_svg]):
            u[self._library_id].savefig(
                out_plot, f"{v}_{self._library_id}{suffix}{suffix_ct}.jpeg")
        for p in fig_gex:
            fig_gex[p][self._library_id].savefig(
                out_plot, f"gex_{p}_{self._library_id}{suffix}"
                f"{suffix_ct}_{'_'.join(goi)}.jpeg")
    self.write(os.path.join(
        out_obj, x + suffix + "_downstream" + obj_ext))



Uninflamed-50403A2




NameError: name 'suffix' is not defined

## By Region

Or any other iteration of objects stored in sub-directories

In [7]:
sc.read_h5ad(out).obs

,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,region,Sample,sample_id,subject_id,run_id,panel_id,slide_id,clinical_block,description,date_hybridization,date_sectioned,location,inflammation,stricture,Condition,tissue_region_crop,Age,Diagnosis,Disease_Status,Hispanic,Race,Sex,Unnamed: 29,assay_protein,cell_filter_ncounts,cell_filter_ngene,cell_filter_pmt,cell_labels,col_batch,col_cell_type,col_condition,col_gene_symbols,col_num_umis,col_sample_id,col_segment,col_subject,custom_thresholds,figsize,file_path,gene_filter_ncell,gene_filter_ncounts,kws_scale,label_res1pt5_dist0_npc30,leiden_res1pt5_dist0_npc30,log1p_n_genes_by_counts,log1p_total_counts,log_counts,method_norm,n_counts,n_genes,n_genes_by_counts,nucleus_count,out_file,segmentation,z_level
176,aaddfnmb-1,273,0,0,0,0,273.0,129.869380,41.092189,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,177,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},BEST2+ Goblet cell,0,4.762174,5.613128,5.609472,log,273.0,116,116,1.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
177,aaddibpm-1,211,0,0,0,0,211.0,86.067816,29.803126,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,178,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},BEST2+ Goblet cell,3,4.553877,5.356586,5.351858,log,211.0,94,94,1.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
179,aaddkhpk-1,181,0,0,0,0,181.0,63.354221,23.481251,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,180,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},ILC1,8,4.736198,5.204007,5.198497,log,181.0,113,113,2.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
185,aadfalag-1,96,0,0,0,0,96.0,76.314065,36.034689,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,186,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},CLP,11,4.060443,4.574711,4.564348,log,96.0,57,57,

In [27]:
import spatialdata



'spatialdata_attrs'

In [23]:
self.rna.uns[ATTRS_KEY]

{'region': 'cell_circles', 'region_key': 'region', 'instance_key': 'cell_id'}

In [25]:
ann.uns[ATTRS_KEY]

{'instance_key': 'cell_id',
 'region': array(['cell_circles'], dtype=object),
 'region_key': 'region'}

In [20]:
REGION_KEY="region"
REGION_KEY_KEY="region_key"
ATTRS_KEY = "spatialdata_attrs"

In [16]:
        attr = {'instance_key': 'cell_id', 'region': np.array(['cell_circles'], dtype=object), 'region_key': 'region'}
        data = self.rna

        expected_regions = attr[REGION_KEY] if isinstance(attr[REGION_KEY], list) else [attr[REGION_KEY]]
        found_regions = data.obs[attr[REGION_KEY_KEY]].unique().tolist()


In [17]:
len(set(expected_regions).symmetric_difference(set(found_regions))) > 0

TypeError: unhashable type: 'numpy.ndarray'

In [19]:
self._spatial_key

'spatial'

In [34]:
ann = sc.read_h5ad(out)

In [40]:
ann.obs

,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,region,Sample,sample_id,subject_id,run_id,panel_id,slide_id,clinical_block,description,date_hybridization,date_sectioned,location,inflammation,stricture,Condition,tissue_region_crop,Age,Diagnosis,Disease_Status,Hispanic,Race,Sex,Unnamed: 29,assay_protein,cell_filter_ncounts,cell_filter_ngene,cell_filter_pmt,cell_labels,col_batch,col_cell_type,col_condition,col_gene_symbols,col_num_umis,col_sample_id,col_segment,col_subject,custom_thresholds,figsize,file_path,gene_filter_ncell,gene_filter_ncounts,kws_scale,label_res1pt5_dist0_npc30,leiden_res1pt5_dist0_npc30,log1p_n_genes_by_counts,log1p_total_counts,log_counts,method_norm,n_counts,n_genes,n_genes_by_counts,nucleus_count,out_file,segmentation,z_level
176,aaddfnmb-1,273,0,0,0,0,273.0,129.869380,41.092189,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,177,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},BEST2+ Goblet cell,0,4.762174,5.613128,5.609472,log,273.0,116,116,1.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
177,aaddibpm-1,211,0,0,0,0,211.0,86.067816,29.803126,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,178,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},BEST2+ Goblet cell,3,4.553877,5.356586,5.351858,log,211.0,94,94,1.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
179,aaddkhpk-1,181,0,0,0,0,181.0,63.354221,23.481251,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,180,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},ILC1,8,4.736198,5.204007,5.198497,log,181.0,113,113,2.0,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50403A2,new,0.0
185,aadfalag-1,96,0,0,0,0,96.0,76.314065,36.034689,cell_circles,Uninflamed-50403A2,50403A2,50403,CHO-011,TUQ97N,10589,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,2024-05-07 00:00:00.000,2024-03-21 00:00:00.000,Terminal Ileum,uninflamed,no,uninflamed,mucosa,33,Crohn's Disease,Active,YES,white,female,nan,None,"[15, None]","[3, None]",None,186,Sample,manual_res0pt5_dist0pt5_npc30,None,gene_symbols,None,Sample,region,Patient,None,"(15, 15)",/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050,"[3, None]","[3, None]",{},CLP,11,4.060443,4.574711,4.564348,log,96.0,57,57,

In [52]:
import spatialdata_io as sdio
import spatialdata

x = metadata.index.values[0]
adata = sdio.xenium(metadata.loc[x][col_f], n_jobs=12)
# adata = spatialdata.deepcopy(adata)
adata

INFO     reading                                                                                                   
         /mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050
         /cell_feature_matrix.h5                                                                                   


In [54]:
sdata_table = adata.tables["table"].copy()

In [58]:
adata.tables["table"] = sdata_table.copy()

In [6]:
ann = sc.read_h5ad(os.path.join(out_obj, x + obj_ext))

In [28]:
ann_parsed = spatialdata.models.TableModel.parse(ann.copy())

In [44]:
adata

SpatialData object
├── Images
│     └── 'morphology_focus': DataTree[cyx] (5, 47706, 37035), (5, 23853, 18517), (5, 11926, 9258), (5, 5963, 4629), (5, 2981, 2314)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (47706, 37035), (23853, 18517), (11926, 9258), (5963, 4629), (2981, 2314)
│     └── 'nucleus_labels': DataTree[yx] (47706, 37035), (23853, 18517), (11926, 9258), (5963, 4629), (2981, 2314)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 11) (3D points)
└── Shapes
      ├── 'cell_boundaries': GeoDataFrame shape: (212695, 1) (2D shapes)
      ├── 'cell_circles': GeoDataFrame shape: (212695, 2) (2D shapes)
      └── 'nucleus_boundaries': GeoDataFrame shape: (209522, 1) (2D shapes)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), cell_labels (Labels), nucleus_labels (Labels), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes), nucleus_boundaries (Shapes)

In [49]:
adata.labels["cell_labels"]

DataTree('None', parent=None)
├── DataTree('scale0')
│       Dimensions:  (y: 47706, x: 37035)
│       Coordinates:
│         * y        (y) float64 382kB 0.5 1.5 2.5 3.5 ... 4.77e+04 4.77e+04 4.771e+04
│         * x        (x) float64 296kB 0.5 1.5 2.5 3.5 ... 3.703e+04 3.703e+04 3.703e+04
│       Data variables:
│           image    (y, x) uint32 7GB dask.array<chunksize=(5792, 5792), meta=np.ndarray>
├── DataTree('scale1')
│       Dimensions:  (y: 23853, x: 18517)
│       Coordinates:
│         * y        (y) float64 191kB 1.0 3.0 5.0 7.0 ... 4.77e+04 4.77e+04 4.77e+04
│         * x        (x) float64 148kB 1.0 3.0 5.0 7.0 ... 3.703e+04 3.703e+04 3.703e+04
│       Data variables:
│           image    (y, x) uint32 2GB dask.array<chunksize=(5792, 5792), meta=np.ndarray>
├── DataTree('scale2')
│       Dimensions:  (y: 11926, x: 9258)
│       Coordinates:
│         * y        (y) float64 95kB 2.0 6.0 10.0 14.0 ... 4.77e+04 4.77e+04 4.77e+04
│         * x        (x) float64 74kB 2.0 6.0 10.0 ... 3.702e+04 3.703e+04 3.703e+04
│       Data variables:
│           image    (y, x) uint32 442MB dask.array<chunksize=(5792, 5792), meta=np.ndarray>
├── DataTree('scale3')
│       Dimensions:  (y: 5963, x: 4629)
│       Coordinates:
│         * y        (y) float64 48kB 4.0 12.0 20.0 ... 4.769e+04 4.769e+04 4.77e+04
│         * x        (x) float64 37kB 4.0 12.0 20.0 ... 3.701e+04 3.702e+04 3.703e+04
│       Data variables:
│           image    (y, x) uint32 110MB dask.array<chunksize=(5963, 4629), meta=np.ndarray>
└── DataTree('scale4')
        Dimensions:  (y: 2981, x: 2314)
        Coordinates:
          * y        (y) float64 24kB 8.002 24.01 40.01 ... 4.767e+04 4.768e+04 4.77e+04
          * x        (x) float64 19kB 8.002 24.01 40.01 ... 3.701e+04 3.703e+04
        Data variables:
            image    (y, x) uint32 28MB dask.array<chunksize=(2981, 2314), meta=np.ndarray>

In [59]:
self = cr.Spatial(adata, library_id=x, **kws_init)



<<< INITIALIZING SPATIAL CLASS OBJECT >>>



<<< LOADING OBJECT >>>


Counts: Initial

'SpatialData' object has no attribute 'n_obs'

Error printing counts SpatialData object
├── Images
│     └── 'morphology_focus': DataTree[cyx] (5, 47706, 37035), (5, 23853, 18517), (5, 11926, 9258), (5, 5963, 4629), (5, 2981, 2314)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (47706, 37035), (23853, 18517), (11926, 9258), (5963, 4629), (2981, 2314)
│     └── 'nucleus_labels': DataTree[yx] (47706, 37035), (23853, 18517), (11926, 9258), (5963, 4629), (2981, 2314)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 11) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (212695, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (212695, 2) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (209522, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (212695, 469)
with coordinate systems:
    ▸ 'global', with elements:
        morpholog

In [64]:
r = "_mucosa"
sub_d = "mucosa"

self.update_from_h5ad(os.path.join(out_obj, sub_dir_parent, sub_d, x + r + obj_ext))  # update with prior preprocessing



<<< RESTORING SINGLE-SAMPLE FROM h5ad >>>



In [65]:
# Iterate Regions
for r, sub_d in zip(suffixes, sub_dirs):
    fig_central, fig_neigh, fig_cooccur, fig_svg = {}, {}, {}, {}
    fig_gex = {"spatial": {}, "heat": {}, "dot": {}}

    # Iterate Samples
    for x in metadata.index.values:
        print(f"\n\n{'=' * 80}\n{x}\n{'=' * 80}\n\n")

        # Load Data
        out = os.path.join(out_obj, sub_dir_parent, sub_d, x + r + obj_ext)
        if not os.path.exists(out):
            print(f"Skipping {x}: {out} doesn't exist!")
            continue

        self = cr.Spatial(metadata.loc[x][col_f], library_id=x, **kws_init)
        self.update_from_h5ad(out)  # update with prior preprocessing
        for j in metadata.dropna(how="all", axis=1):  # add metadata to .obs
            self.rna.obs.loc[:, j] = str(metadata.loc[x][j])
        self.rna.obs.loc[:, col_object] = out  # path for processed object

        # Map Manual Annotations
        annot = f_ann.loc[
            f"{self._library_id}___{col_leiden}_dictionary.xlsx"]
        _ = self.annotate_clusters(
            annot[col_assignment], col_cell_type=col_leiden,
            col_annotation=col_cluster, copy=False)

        # Gene Expression Plots (Only GEX Data)
        goi = list(set(pd.unique(functools.reduce(lambda u, v: u + v, [
            genes[g] for g in genes])) if isinstance(genes, dict) else genes
                    ).intersection(self.rna.var_names))  # gene list
        fig_gex["spatial"][self._library_id] = self.plot_spatial(
            color=goi + [col_cluster], title=f"{self._library_id}{r}")
        fig_gex["dot"][self._library_id] = cr.pl.plot_dot(
            self.rna, "Annotation", genes,
            title=f"{self._library_id}{r}",
            genes_dict_colors=genes_dict_colors, vmin=0, vmax=10,
            percent="right", center=None)
        fig_gex["heat"][self._library_id] = cr.pl.plot_matrix(
            self.rna, "Annotation", genes,
            title=f"{self._library_id}{r}",
            genes_dict_colors=genes_dict_colors, vmin=0, vmax=10,
            percent="right", center=None)

        # Spatial Analyses
        _, fig_central[self._library_id] = self.calculate_centrality(
            figsize=(20, 10))
        _, fig_neigh[self._library_id] = self.calculate_neighborhood(
            figsize=(60, 30))
        _, fig_cooccur[self._library_id] = self.find_cooccurrence(
            figsize=(60, 20), kws_plot=dict(wspace=3))
        _, fig_svg[self._library_id] = self.find_svgs(
            genes=goi, method="moran", n_perms=10, kws_plot=dict(
                legend_fontsize="large"), figsize=(15, 15))

        # Write Plots
        if out_plot is not None:
            for v, u in zip(["central", "neighbor", "cooccur", "svg"], [
                    fig_central, fig_neigh, fig_cooccur, fig_svg]):
                u[self._library_id].savefig(
                    out_plot, sub_d,
                    f"{v}_{self._library_id}{r}{suffix_ct}.jpeg")
            for p in fig_gex:
                fig_gex[p][self._library_id].savefig(
                    out_plot, sub_d, f"gex_{p}_{self._library_id}{r}"
                    f"{suffix_ct}_{'_'.join(goi)}.jpeg")



Uninflamed-50403A2




<<< INITIALIZING SPATIAL CLASS OBJECT >>>

INFO     reading                                                                                                   
         /mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-011/output-XETG00189__0010589__50403A2-TUQ97N-EA__20240513__201050
         /cell_feature_matrix.h5                                                                                   


In [ ]:
data.obs[attr[self.REGION_KEY_KEY]].unique().tolist()

In [28]:
out

'/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Uninflamed-50403A2_mucosa.h5ad'

In [29]:
ann = sc.read("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Uninflamed-50403A2.h5ad")

In [32]:
for i in ['Stricture-50115A2.h5ad',
 'Inflamed-50336B.h5ad',
 'Inflamed-50452B.h5ad',
 'Uninflamed-50403A1.h5ad',
 'Stricture-50403C2.h5ad',
 'Stricture-50445A3.h5ad',
 'Stricture-50403C1.h5ad',
 'Stricture-50452C.h5ad',
 'Stricture-50564A4.h5ad',
 'Uninflamed-50403A2.h5ad',
 'Inflamed-50217A.h5ad',
 'Stricture-50006C.h5ad',
 'Inflamed-50403B.h5ad',
 'Stricture-50217C.h5ad',
 'Uninflamed-50336C.h5ad',
 'Uninflamed-50217B.h5ad',
 'Uninflamed-50452A.h5ad',
 'Stricture-50007B2.h5ad',
 'Inflamed-50006A.h5ad',
 'Stricture-49696A4.h5ad',
 'Stricture-50618B5.h5ad',
 'Uninflamed-50006B.h5ad',
 'Stricture-49471A4.h5ad',
 'Stricture-50336A.h5ad']:
    lib = i.split(".h5ad")[0]
    ann = sc.read("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Uninflamed-50403A2.h5ad")
    ann.obs = ann.obs.drop(ann.obs.columns.intersection(set(['Sample', 'Sample ID', 'file_path', 'n_counts', 'log_counts', 'n_genes',
       'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts',
       'assay_protein', 'col_gene_symbols', 'col_cell_type', 'col_sample_id',
       'col_batch', 'col_subject', 'col_condition', 'col_num_umis',
       'col_segment', 'cell_filter_pmt', 'cell_filter_ncounts',
       'cell_filter_ngene', 'gene_filter_ncell', 'gene_filter_ncounts',
       'custom_thresholds', 'method_norm', 'figsize', 'kws_scale',
       'leiden_res1pt5_dist0_npc30', 'label_res1pt5_dist0_npc30', 'Name',
       'Inflammation', 'bucket_res1pt5_dist0_npc30', 'bin_res1pt5_dist0_npc30',
       'annotation_res1pt5_dist0_npc30', 'n_obs_raw'])), axis=1)
    ann.obs.loc[:, "Sample"] = lib
    for j in metadata.dropna(how="all", axis=1):  # add metadata to .obs
        ann.obs.loc[:, j] = str(metadata.loc[x][j])

['Stricture-50115A2.h5ad',
 'Inflamed-50336B.h5ad',
 'Inflamed-50452B.h5ad',
 'Uninflamed-50403A1.h5ad',
 'Stricture-50403C2.h5ad',
 'Stricture-50445A3.h5ad',
 'Stricture-50403C1.h5ad',
 'Stricture-50452C.h5ad',
 'Stricture-50564A4.h5ad',
 'Uninflamed-50403A2.h5ad',
 'Inflamed-50217A.h5ad',
 'Stricture-50217C_downstream.h5ad',
 'Stricture-50006C.h5ad',
 'Stricture-50336A_downstream.h5ad',
 'Inflamed-50403B.h5ad',
 'Stricture-50217C.h5ad',
 'Uninflamed-50336C.h5ad',
 'Uninflamed-50217B.h5ad',
 'Uninflamed-50452A.h5ad',
 'Stricture-50007B2.h5ad',
 'combined.h5ad',
 'Inflamed-50006A.h5ad',
 'Stricture-49696A4.h5ad',
 'Stricture-50618B5.h5ad',
 'Uninflamed-50006B.h5ad',
 'Stricture-49471A4.h5ad',
 'Inflamed-50336B_downstream.h5ad',
 'Stricture-50336A.h5ad']

In [49]:
 ann = sc.read(f"/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/{lib}.h5ad")

In [61]:
adata = sc.read(f"/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/{lib}.h5ad")

In [82]:
r

'mucosa'

In [87]:
regions = ["mucosa", "submucosa", "myenteric_plexus",
            "smc_circular", "smc_longitudinal", "serosa"]
for r in regions:
   print("\n\n\n")
   for i in ['Stricture-50115A2.h5ad',
   'Inflamed-50336B.h5ad',
   'Inflamed-50452B.h5ad',
   'Uninflamed-50403A1.h5ad',
   'Stricture-50403C2.h5ad',
   'Stricture-50445A3.h5ad',
   'Stricture-50403C1.h5ad',
   'Stricture-50452C.h5ad',
   'Stricture-50564A4.h5ad',
   'Uninflamed-50403A2.h5ad',
   'Inflamed-50217A.h5ad',
   'Stricture-50006C.h5ad',
   'Inflamed-50403B.h5ad',
   'Stricture-50217C.h5ad',
   'Uninflamed-50336C.h5ad',
   'Uninflamed-50217B.h5ad',
   'Uninflamed-50452A.h5ad',
   'Stricture-50007B2.h5ad',
   'Inflamed-50006A.h5ad',
   'Stricture-49696A4.h5ad',
   'Stricture-50618B5.h5ad',
   'Uninflamed-50006B.h5ad',
   'Stricture-49471A4.h5ad',
   'Stricture-50336A.h5ad']:
      lib = i.split(".h5ad")[0]
      fff = f"/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/{r}/{lib}_{r}.h5ad"
      if not os.path.exists(fff):
         continue
      print(fff)
      ann = sc.read(fff)
      ann.obs = ann.obs.drop(ann.obs.columns.intersection(set(['Sample', 'Sample ID', 'Patient', 'Status',
         'Project', 'Location', 'Stricture', 'Inflamed', 'GRID ID',
         'Procedure Date',
         'Project.1', 'Procedure', 'Date Collected',
         'Date Sectioned', 'Date Hybridization', 'Storage 4c', 'Created By',
         'Created', 'Storage Status', 'Location.1', 'Storage Row', 'Storage Col',
         'Checked Out By', 'Condition', 'Name', 'Slide Id', 'Inflammation'])), axis=1)
      ann.obs.loc[:, "Sample"] = lib
      mdcs = ["Sample"]
      for j in metadata.replace("nan", np.nan).dropna(how="all", axis=1):  # add metadata to .obs
         ann.obs.loc[:, j] = str(metadata.loc[lib][j])
         mdcs += [j]
      ann.obs.loc[:, "tissue_region_crop"] = r
      first = ['cell_id', 'transcript_counts', 'control_probe_counts',
         'control_codeword_counts', 'unassigned_codeword_counts',
         'deprecated_codeword_counts', 'total_counts', 'cell_area',
         'nucleus_area', 'region']
      ann.obs = ann.obs[first + mdcs + ["tissue_region_crop"] + list(ann.obs.columns.difference(first + ["tissue_region_crop"] + mdcs))]
      ann.write_h5ad(fff)





/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Inflamed-50336B_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Uninflamed-50403A2_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Inflamed-50217A_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Inflamed-50403B_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Uninflamed-50336C_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Uninflamed-50217B_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Inflamed-50006A_mucosa.h5ad
/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/n

In [67]:
adata.obs.columns.difference(ann.obs.columns)

Index(['Checked Out By', 'Created', 'Created By', 'Date Collected',
       'Date Hybridization', 'Date Sectioned', 'GRID ID', 'Inflamed',
       'Inflammation', 'Location', 'Location.1', 'Name', 'Patient',
       'Procedure', 'Procedure Date', 'Project', 'Project.1', 'Sample ID',
       'Slide Id', 'Status', 'Storage 4c', 'Storage Col', 'Storage Row',
       'Storage Status', 'Stricture'],
      dtype='object')

In [86]:
sc.read("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/archive_pre_rewrite_md/Inflamed-50336B_mucosa.h5ad").obs

,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,region,z_level,nucleus_count,cell_labels,Sample,Sample ID,Patient,Status,Slide Id,Project,Location,Stricture,GRID ID,Inflamed,Procedure Date,Age,Sex,Race,Hispanic,Diagnosis,Project.1,Procedure,Disease_Status,Date Collected,Date Sectioned,Date Hybridization,Storage 4c,Created By,Created,Storage Status,Location.1,Storage Row,Storage Col,Checked Out By,out_file,Condition,file_path,n_counts,log_counts,n_genes,n_genes_by_counts,log1p_n_genes_by_counts,log1p_total_counts,assay_protein,col_gene_symbols,col_cell_type,col_sample_id,col_batch,col_subject,col_condition,col_num_umis,col_segment,cell_filter_pmt,cell_filter_ncounts,cell_filter_ngene,gene_filter_ncell,gene_filter_ncounts,custom_thresholds,method_norm,figsize,kws_scale,leiden_res1pt5_dist0_npc30,label_res1pt5_dist0_npc30,leiden_res0pt5_dist0pt5_npc30,label_res0pt5_dist0pt5_npc30,leiden_res0pt75_dist0pt3_npc30,label_res0pt75_dist0pt3_npc30,annotation_res1pt5_dist0_npc30,bin_res1pt5_dist0_npc30,bucket_res1pt5_dist0_npc30,CDKN1A_threshold2,TP53_threshold2,PLAUR_threshold2,CDKN1A_TP53_threshold2,CDKN1A_PLAUR_threshold2,TP53_PLAUR_threshold2,CDKN1A_TP53_PLAUR_threshold2,IL7R_threshold2,KLRB1_threshold2,RORC_threshold2,IL7R_KLRB1_threshold2,IL7R_RORC_threshold2,KLRB1_RORC_threshold2,IL7R_KLRB1_RORC_threshold2
0,aaaaacoc-1,120,0,0,0,0,120.0,72.701565,28.132345,cell_circles,0.0,2.0,1,Inflamed-50336B,50336B,50336,Available,11044.0,scRNA & Spatial,Terminal Ileum,no,1011328,inflamed,2023-11,24,male,white,NO,Crohn's Disease,Helmsley-Senescence,Resection,Active,2024.11.27,2024.03.21,nan,YES,lorena tavares,15-Apr-2024,Not in storage,nan,nan,nan,nan,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50336B,Inflamed,/mnt/cho_lab/bbdata1/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,120.0,4.787492,49,49,3.912023,4.795791,None,gene_symbols,manual_res1pt5_dist0_npc30,Sample,Sample,Patient,None,None,region,None,"[15, None]","[3, None]","[3, None]","[3, None]",None,log,"(15, 15)",{},3,Monocytes,6,Monocyte,7,Monocytes,Monocyte,Monocyte,Myeloid,False,False,False,Other,Other,Other,Other,False,False,False,Other,Other,Other,Other
1,aaaabdap-1,66,0,0,0,0,66.0,44.388595,26.235782,cell_circles,0.0,1.0,2,Inflamed-50336B,50336B,50336,Available,11044.0,scRNA & Spatial,Terminal Ileum,no,1011328,inflamed,2023-11,24,male,white,NO,Crohn's Disease,Helmsley-Senescence,Resection,Active,2024.11.27,2024.03.21,nan,YES,lorena tavares,15-Apr-2024,Not in storage,nan,nan,nan,nan,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50336B,Inflamed,/mnt/cho_lab/bbdata1/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,66.0,4.189655,45,45,3.828641,4.204693,None,gene_symbols,manual_res1pt5_dist0_npc30,Sample,Sample,Patient,None,None,region,None,"[15, None]","[3, None]","[3, None]","[3, None]",None,log,"(15, 15)",{},11,BEST2+ Goblet cell,1,DC,1,BEST2+ Goblet cell,Epithelial,Epithelial,Epithelial,False,False,False,Other,Other,Other,Other,False,False,False,Other,Other,Other,Other
3,aaaajjap-1,97,0,0,0,0,97.0,61.502815,30.841720,cell_circles,0.0,1.0,4,Inflamed-50336B,50336B,50336,Available,11044.0,scRNA & Spatial,Terminal Ileum,no,1011328,inflamed,2023-11,24,male,white,NO,Crohn's Disease,Helmsley-Senescence,Resection,Active,2024.11.27,2024.03.21,nan,YES,lorena tavares,15-Apr-2024,Not in storage,nan,nan,nan,nan,/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/Inflamed-50336B,Inflamed,/mnt/cho_lab/bbdata1/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,97.0,4.574711,40,40,3.713572,4.584968,None,gene_symbols,manual_res1pt5_dist0_npc30,Sample,Sample,Patient,None,None,region,None,"[15, None]","[3, None]","[3, None]","[3, None]",None,log,"(15, 15)",{},3,Monocytes,

In [85]:
sc.read("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/objects_cropped/mucosa/Inflamed-50336B_mucosa.h5ad").obs

""
0
1
2
3
4
...
371090
371091
371092
371093


In [81]:
ann.obs

,Sample,sample_id,subject_id,run_id,panel_id,slide_id,clinical_block,description,date_hybridization,date_sectioned,location,inflammation,stricture,Condition,tissue_region_crop
0,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
1,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
2,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
3,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
4,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371090,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
371091,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
371092,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
371093,Inflamed-50336B,50336B,50336,CHO-010,TUQ97N,11044,NO,/mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011044__50336B-TUQ97N-EA__20240422__175051,nan,2024-03-21 00:00:00.000,Terminal Ileum,inflamed,no,inflamed,mucosa
